# Applied Data Science Capstone Assignment2

## Introduction of Assingment2

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

## Part1 Create DataFrame Based on Wikipedia postcode table

**Import pandas and get table from Wikipedia**

Frist we will get data from wikipedia by following URL:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

I am a chinese, due to The Great Firewall, I am not able to access Wikipedia, I have to ask my friend to download the table(using pd.read_html() method then export to csv) for me then I could read from the csv instead.

In [1]:
import pandas as pd

In [2]:
data = 'canada_postcode.csv'
postcode_data = pd.read_csv(data)
postcode_data.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True) 
postcode_data.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


**Count rows**

In [3]:
postcode_data.count()

Postcode        288
Borough         288
Neighborhood    288
dtype: int64

**Ignore cells with "Not assigned'**

In [4]:
filter_postcodes = postcode_data[postcode_data['Borough'] != 'Not assigned']

In [5]:
filter_postcodes.count()

Postcode        211
Borough         211
Neighborhood    211
dtype: int64

**Combine Neighbourhoods according to Postcode and Borough**

In [6]:
filter_postcodes = filter_postcodes.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join).reset_index()

In [8]:
filter_postcodes.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
filter_postcodes.loc[filter_postcodes['Neighborhood'] == 'Not assigned', 'Neighborhood'] = 'Queen\'s Park'

In [9]:
filter_postcodes[filter_postcodes['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [10]:
filter_postcodes.count()

Postcode        103
Borough         103
Neighborhood    103
dtype: int64

In [11]:
print('postcodes DataFrame now has %d rows.' %filter_postcodes.count()[0])

postcodes DataFrame now has 103 rows.


# Part2 Merge geodata dataframe to the postcode dataframe

In this part, I will get latitude and longitude data foreach postcode and then add these data to Part1 filter_postcode dataframe.

**Read PostCode Latitude and Longitude csv file**

In [12]:
lag_long = pd.read_csv('http://cocl.us/Geospatial_data')

In [13]:
lag_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**use concat method to combine *lag_long* and *filter_postcode* dataframe** 

In [14]:
finaldf = pd.concat([filter_postcodes,lag_long],axis=1,join='outer').drop(['Postcode','Postal Code'],axis=1)
finaldf.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
print(finaldf.shape)

(103, 4)


# Part3 Analysis

**Import the necessary libraries for analysing**

In [16]:
import numpy as np 

# import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # Library to handle JSON files

from geopy.geocoders import Nominatim # Convert an address into latitude and longitude values

import requests # Library to handle requests
from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # Map rendering library

print('Libraries imported.')

Libraries imported.


**Get borough and neighborhood numbers**

In [17]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(finaldf['Borough'].unique()),
        finaldf.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


**Use geopy library to get the latitude and longitude values of Toronto.**

In [18]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


**Create a map of Toronto to visualize the neighbourhoods**

In [19]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(finaldf['Latitude'], finaldf['Longitude'], finaldf['Borough'], finaldf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Define Foursquare API**

In [20]:
CLIENT_ID = 'TSM3JV4NA3T0JMVKH321QF0KMSWUEGE4KCBGDKZYZP3MHMFT' # your Foursquare ID
CLIENT_SECRET = 'OHEDQH1YJDTEREPLCJOKDMYV4SRIKZC3UES4Q253Y4QQWYTQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TSM3JV4NA3T0JMVKH321QF0KMSWUEGE4KCBGDKZYZP3MHMFT
CLIENT_SECRET:OHEDQH1YJDTEREPLCJOKDMYV4SRIKZC3UES4Q253Y4QQWYTQ


**Explore Neighbourhoods in Toronto**

In [21]:
import time

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        time.sleep(0.5)    
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    time.sleep(0.5)
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(names=finaldf['Neighborhood'],
                                latitudes=finaldf['Latitude'],
                                longitudes=finaldf['Longitude'],)

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

**Check new dataframe**

In [24]:
toronto_venues.shape

(2216, 7)

In [25]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


**check how many venues were returned for each neighborhood**

In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",8,8,8,8,8,8
"Alderwood,Long Branch",10,10,10,10,10,10
"Bathurst Manor,Downsview North,Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",22,22,22,22,22,22
Berczy Park,56,56,56,56,56,56


**find out how many unique categories can be curated from all the returned venues**

In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


**Analyze Each Neiborhood**

In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [29]:
toronto_onehot.shape

(2216, 269)

**let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.010000,0.000000,0.030000,0.000000,0.00

In [31]:
toronto_grouped.shape

(100, 269)

**print top 5 most common venues for each neighbourhood**

In [32]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2       Steakhouse  0.04
3  Thai Restaurant  0.03
4       Restaurant  0.03


----Agincourt----
                       venue  freq
0             Clothing Store   0.2
1                     Lounge   0.2
2             Breakfast Spot   0.2
3               Skating Rink   0.2
4  Latin American Restaurant   0.2


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
               venue  freq
0               Park   0.5
1         Playground   0.5
2        Yoga Studio   0.0
3        Men's Store   0.0
4  Mobile Phone Shop   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.25
1              Pharmacy  0.12
2           Pizza Place  0.12
3  Fast Food Restaurant  0.12
4        Sandwich Place  0.12


----Alderwood,Long Branch----
                venue  freq
0         

**add common venues into dataframe**

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Restaurant,Asian Restaurant,Thai Restaurant,Burger Joint,Bakery,Bar,Salad Place
1,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fried Chicken Joint,Pharmacy,Pizza Place,Fast Food Restaurant,Beer Store,Sandwich Place,Doner Restaurant,Dim Sum Restaurant,Diner
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Dance Studio,Pub,Athletics & Sports,Skating Rink,Pharmacy,Gym,Dessert Shop


**Run k-means to cluster the neiborhood into 5 clusters.**

In [35]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

**create a new dataframe including neighbourhood, latitude, longitude and common venue data**

In [36]:
###Test

In [38]:
#neighborhoods_venues_sorted.set_index('Neighborhood')
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Restaurant,Asian Restaurant,Thai Restaurant,Burger Joint,Bakery,Bar,Salad Place
1,0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
3,0,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fried Chicken Joint,Pharmacy,Pizza Place,Fast Food Restaurant,Beer Store,Sandwich Place,Doner Restaurant,Dim Sum Restaurant,Diner
4,0,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Dance Studio,Pub,Athletics & Sports,Skating Rink,Pharmacy,Gym,Dessert Shop


In [39]:
finaldf.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [40]:
toronto_merged = finaldf
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Department Store
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0.0,Moving Target,Bar,Women's Store,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dim Sum Restaurant
2,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Mexican Restaurant,Intersection,Rental Car Location,Electronics Store,Pizza Place,Spa,Breakfast Spot,Medical Center,Diner,Discount Store
3,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Lounge,Bakery,Bank,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Fried Chicken Joint,Hakka Restaurant,Dumpling Restaurant


In [41]:
toronto_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Department Store
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0.0,Moving Target,Bar,Women's Store,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dim Sum Restaurant
2,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Mexican Restaurant,Intersection,Rental Car Location,Electronics Store,Pizza Place,Spa,Breakfast Spot,Medical Center,Diner,Discount Store
3,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Lounge,Bakery,Bank,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Fried Chicken Joint,Hakka Restaurant,Dumpling Restaurant
5,Scarborough,Scarborough Village,43.744734,-79.239476,3.0,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
6,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0.0,Discount Store,Department Store,Coffee Shop,Dumpling Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
7,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,0.0,Bakery,Bus Line,Park,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,Doner Restaurant,Discount Store,Dog Run
8,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,0.0,Motel,Movie Theater,American Restaurant,Women's Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
9,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,0.0,College Stadium,General Entertainment,Skating Rink,Café,Comic Shop,Concert Hall,Ethiopian Restaurant,Colombian Restaurant,Empanada Restaurant,Electronics Store


In [42]:
toronto_merged.dropna(inplace=True)
toronto_merged = toronto_merged.reset_index(drop=True)

In [43]:
toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Department Store
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0.0,Moving Target,Bar,Women's Store,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dim Sum Restaurant
2,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Mexican Restaurant,Intersection,Rental Car Location,Electronics Store,Pizza Place,Spa,Breakfast Spot,Medical Center,Diner,Discount Store
3,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Lounge,Bakery,Bank,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Fried Chicken Joint,Hakka Restaurant,Dumpling Restaurant


In [44]:
toronto_merged[['Cluster Labels']]=toronto_merged[['Cluster Labels']].astype(np.int64)

In [45]:
toronto_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Print Shop,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Department Store
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0,Moving Target,Bar,Women's Store,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dim Sum Restaurant
2,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0,Mexican Restaurant,Intersection,Rental Car Location,Electronics Store,Pizza Place,Spa,Breakfast Spot,Medical Center,Diner,Discount Store
3,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,Scarborough,Cedarbrae,43.773136,-79.239476,0,Lounge,Bakery,Bank,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Fried Chicken Joint,Hakka Restaurant,Dumpling Restaurant
5,Scarborough,Scarborough Village,43.744734,-79.239476,3,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
6,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0,Discount Store,Department Store,Coffee Shop,Dumpling Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
7,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,0,Bakery,Bus Line,Park,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,Doner Restaurant,Discount Store,Dog Run
8,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,0,Motel,Movie Theater,American Restaurant,Women's Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
9,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,0,College Stadium,General Entertainment,Skating Rink,Café,Comic Shop,Concert Hall,Ethiopian Restaurant,Colombian Restaurant,Empanada Restaurant,Electronics Store


In [46]:
###EndTest

**visualize the clusters**

In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Examine Clusters**

examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",Fast Food Restaurant,Print Shop,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Department Store
1,"Highland Creek,Rouge Hill,Port Union",Moving Target,Bar,Women's Store,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dim Sum Restaurant
2,"Guildwood,Morningside,West Hill",Mexican Restaurant,Intersection,Rental Car Location,Electronics Store,Pizza Place,Spa,Breakfast Spot,Medical Center,Diner,Discount Store
3,Woburn,Coffee Shop,Korean Restaurant,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,Cedarbrae,Lounge,Bakery,Bank,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Fried Chicken Joint,Hakka Restaurant,Dumpling Restaurant
6,"East Birchmount Park,Ionview,Kennedy Park",Discount Store,Department Store,Coffee Shop,Dumpling Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
7,"Clairlea,Golden Mile,Oakridge",Bakery,Bus Line,Park,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,Doner Restaurant,Discount Store,Dog Run
8,"Cliffcrest,Cliffside,Scarborough Village West",Motel,Movie Theater,American Restaurant,Women's Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
9,"Birch Cliff,Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Comic Shop,Concert Hall,Ethiopian Restaurant,Colombian Restaurant,Empanada Restaurant,Electronics Store
10,"Dorset Park,Scarborough Town Centre,Wexford He...",Indian Restaurant,Light Rail Station,Pet Store,Vietnamese Restaurant,Brewery,Chinese Restaurant,Drugstore,Discount Store,Dog Run,Doner Restaurant


**Cluster 1**

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Downsview Central,Food Truck,Baseball Field,Women's Store,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant
89,"Humber Bay,King's Mill Park,Kingsway Park Sout...",Baseball Field,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Festival
94,"Emery,Humberlea",Baseball Field,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Festival


**Cluster 2**

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
21,York Mills West,Park,Bank,Convenience Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
23,Parkwoods,Park,Construction & Landscaping,Food & Drink Shop,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
28,"CFB Toronto,Downsview East",Park,Airport,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
38,East Toronto,Park,Coffee Shop,Convenience Store,Ethiopian Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop,Dumpling Restaurant
42,Lawrence Park,Park,Swim School,Bus Line,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dim Sum Restaurant
48,Rosedale,Park,Playground,Trail,Ethiopian Restaurant,Empanada Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store
72,Caledonia-Fairbanks,Park,Market,Women's Store,Fast Food Restaurant,Golf Course,Ethiopian Restaurant,Empanada Restaurant,Greek Restaurant,Electronics Store,Eastern European Restaurant
77,"Downsview,North Park,Upwood Park",Park,Basketball Court,Bakery,Construction & Landscaping,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
88,"The Kingsway,Montgomery Road,Old Mill North",River,Pool,Park,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Ethiopian Restaurant,Deli / Bodega


**Cluster 3**

In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
46,"Moore Park,Summerhill East",Restaurant,Playground,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore


**Cluster 4**

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
93,Humber Summit,Pizza Place,Empanada Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
97,"Kingsview Village,Martin Grove Gardens,Richvie...",Park,Pizza Place,Mobile Phone Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store,Donut Shop
